### 필요한 모듈 설치

In [47]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
# !pip install torch

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp38-cp38-manylinux1_x86_64.whl (3.0 MB)
ERROR: kobert 0.2.3 has requirement transformers>=4.8.1, but you'll have transformers 3.0.2 which is incompatible.
ERROR: azureml-automl-dnn-nlp 1.40.0 has requirement transformers<=4.5.1,>=4.1.0, but you'll have transformers 3.0.2 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-6soayqks
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-6soayqks
     |████████████████████████████████| 4.5 MB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 99.6 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 6.6 MB 100.1 MB/s eta 0:00:01
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15662 sha256=8add8ddcdc215d81a23691b92faf7566a9511a85e4ddc8b24d9b4b116a285d62
  Stored in directory: /tmp/pip-ephem-wheel-cache-q_p_mr41/wheels/bf/5f/74/81bf3a1332130eb6629ecf58876a8746b77021e7d7b0638e91
Successfully built kobert
ERROR: datasets 1.8.0 has requirement huggingface-hub<0.1.0, but you'll have huggingface-hub 0.5.1 which is incompatible.
ERROR: datasets 1.8.0 has requirement tqdm<4.50.0,>=4.27, but you'll have tqdm 4.62.3 whi

In [20]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [21]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [22]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [23]:
#GPU 사용
device = torch.device("cuda:0")

In [24]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/adminuser/notebooks/.cache/kobert_v1.zip
using cached model. /home/adminuser/notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


### 데이터 전처리

In [29]:
# 구글 드라이브에서 데이터 불러오기

import pandas as pd
chatbot_data = pd.read_excel('./한국어_단발성_대화_데이터셋.xlsx', engine='openpyxl') 

In [30]:
chatbot_data.head()

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0


In [10]:
pip install xlrd # excel

     |████████████████████████████████| 96 kB 1.9 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install openpyxl #excel

     |████████████████████████████████| 242 kB 32.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [31]:
chatbot_data.sample(10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
31865,연아양~오늘도 연아양을 응원하러 들렀습니다.!!,행복,NaN,NaN,NaN,NaN,NaN
2938,이런 내일부터 일하는데 바들바들 떨준비해야긋넹....다들 감기조심하세요!!,공포,NaN,NaN,NaN,NaN,NaN
9963,부인을 언제든지 차 바꾸는것처럼 바꿀수있는능력과 재력이 부럽다,놀람,NaN,NaN,NaN,NaN,NaN
16248,그냥 쳐먹어 먹을거 없어,분노,NaN,NaN,NaN,NaN,NaN
16169,닭년도 최순실만 안걸렸으면 지 아버지처럼 계속 해먹을 생각이었겠지,분노,NaN,NaN,NaN,NaN,NaN
27097,성형을 한들 흉이아니에요,중립,NaN,NaN,NaN,NaN,NaN
21992,"태생이 그런 사람이라, 그렇게 살아와서, 사람 쉽게 못고쳐진다는 말이 있듯이 바뀌...",슬픔,NaN,NaN,NaN,NaN,NaN
27481,근데 피디는 개웃기다,행복,NaN,NaN,NaN,NaN,NaN
25359,네이마르 척추샷 빠 수니가~,중립,NaN,NaN,NaN,NaN,NaN
11696,미친것들!!,분노,NaN,NaN,NaN,NaN,NaN


In [32]:
#  NaN 컬럼들 없애주고, 7가지의 감정 단어를 숫자로 변경
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [33]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [34]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['최악의 상황중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [35]:
#  Train, test 나누기
#train & test 데이터로 나누기 -> 0.25 = 4:1 로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

28945
9649


In [36]:
# 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 해주기 위한 class 
# BERTSentenceTransform 모듈

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [37]:
# 하이퍼 파라미터 설정
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [38]:
# 클래스에서 정의한 class로 토큰화, 패딩 

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/adminuser/notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [39]:
# 토큰화, 패딩이 잘 됐는지 보기
# 첫 번째는 패딩된 시퀀스, 두 번째는 길이와 타입에 대한 내용, 세 번재는 어텐션 마스크 시퀀스
# 어텐션 마스크 시퀀스 :1로 패딩된 값들은 연산할 필요가 없기 때문에 연산을 하지 않아도 된다고 알려주는 데이터

data_train[0]

(array([   2, 1189,  517, 6188, 7245, 7063,  517,  463, 3486, 7836, 5966,
        1698,  517, 6188, 7245, 7063,  517,  463, 1281, 7870, 1801, 6885,
        7088, 5966, 1698, 5837, 5837,  517, 6188, 7245, 6398, 6037, 7063,
         517,  463,  517,  463,  517,  364,  517,  364,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(42, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 5)

In [40]:
# torch 형식의 dataset 만들어주기

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [41]:
# koBERT 학습모델 만들기
# 다중분류할 클래스 수 만큼 num_classes 변수를 수정 (7가지의 class 분류)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                              attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [42]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [43]:
#모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_14031/2869194262.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9736509323120117 train acc 0.1875
epoch 1 batch id 201 loss 1.334525227546692 train acc 0.28910136815920395
epoch 1 batch id 401 loss 1.167905330657959 train acc 0.3818968204488778
epoch 1 train acc 0.3964846286196598


/tmp/ipykernel_14031/2869194262.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/151 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5564941208271388


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.42449015378952026 train acc 0.84375
epoch 5 batch id 201 loss 0.3943263292312622 train acc 0.7867692786069652
epoch 5 batch id 401 loss 0.5472188591957092 train acc 0.7976153366583542
epoch 5 train acc 0.7985042526944552


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 5 test acc 0.5552925226381944


In [44]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴져요.")

using cached model. /home/adminuser/notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [46]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 너무 졸려요
>> 입력하신 내용에서 슬픔이 느껴져요.


하고싶은 말을 입력해주세요 : 배가 고파요
>> 입력하신 내용에서 슬픔이 느껴져요.


하고싶은 말을 입력해주세요 : 공부 하기 싫어요 !!
>> 입력하신 내용에서 분노가 느껴져요.


하고싶은 말을 입력해주세요 : 공부 하기 싫어
>> 입력하신 내용에서 슬픔이 느껴져요.


하고싶은 말을 입력해주세요 : 배 고파 !!
>> 입력하신 내용에서 슬픔이 느껴져요.


하고싶은 말을 입력해주세요 : 배고파 !!!!!!!
>> 입력하신 내용에서 슬픔이 느껴져요.


하고싶은 말을 입력해주세요 : 졸려 !!!!!!!!!!
>> 입력하신 내용에서 행복이 느껴져요.


하고싶은 말을 입력해주세요 : 집에 언제가지 ~
>> 입력하신 내용에서 공포가 느껴져요.


하고싶은 말을 입력해주세요 : 집에 가고 싶다 ~~~~^^
>> 입력하신 내용에서 행복이 느껴져요.


하고싶은 말을 입력해주세요 : 집에 가고 싶다 !!!!!!!
>> 입력하신 내용에서 슬픔이 느껴져요.


하고싶은 말을 입력해주세요 : 개짜증나
>> 입력하신 내용에서 슬픔이 느껴져요.


하고싶은 말을 입력해주세요 : 분노
>> 입력하신 내용에서 분노가 느껴져요.


하고싶은 말을 입력해주세요 : 개짱나
>> 입력하신 내용에서 행복이 느껴져요.


하고싶은 말을 입력해주세요 : 꺼져
>> 입력하신 내용에서 분노가 느껴져요.


하고싶은 말을 입력해주세요 : 우와
>> 입력하신 내용에서 놀람이 느껴져요.


하고싶은 말을 입력해주세요 : 허걱
>> 입력하신 내용에서 놀람이 느껴져요.


하고싶은 말을 입력해주세요 : 오잉?
>> 입력하신 내용에서 놀람이 느껴져요.


하고싶은 말을 입력해주세요 : 짜증나게 하지마
>> 입력하신 내용에서 혐오가 느껴져요.


하고싶은 말을 입력해주세요 : 겁나 좋아
>> 입력하신 내용에서 놀람이 느껴져요.


하고싶은 말을 입력해주세요 : 완전 좋아
>> 입력하신 내용에서 행복이 느껴져요.


하